In [79]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.style as style # for styling the graphs
#_______________________________________________________________________________________________________________
# style.available (to know the available list of styles)
style.use('ggplot') # chosen style
plt.rc('xtick',labelsize=13) # to globally set the tick size
plt.rc('ytick',labelsize=13) # to globally set the tick size
# To print multiple outputs together
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
# Change column display number during print
pd.set_option('display.max_columns', 500)
# Ignore warnings
import warnings
warnings.filterwarnings('ignore')
# To display float with 2 decimal, avoid scientific printing
pd.options.display.float_format = '{:.2f}'.format
import seaborn as sns
import warnings
import sweetviz
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, precision_recall_curve, auc, roc_auc_score, roc_curve, recall_score, classification_report
from sklearn.metrics import roc_auc_score ,mean_squared_error,accuracy_score,classification_report,roc_curve,confusion_matrix,precision_score,f1_score
import itertools
from sklearn.model_selection import GridSearchCV, cross_val_score
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier
from sklearn.model_selection import KFold

In [80]:
path='./Train.csv'
path1='./Test.csv'

In [81]:
train=pd.read_csv(path)
train

,Team1,Team2,Stadium,HostCountry,Team1_Venue,Team2_Venue,Team1_Innings,Team2_Innings,MonthOfMatch,MatchWinner
0,5,4,37,4,Home,Away,Second,First,Dec,4
1,1,14,84,7,Neutral,Neutral,First,Second,Sep,1
2,9,15,47,9,Home,Away,First,Second,Feb,9
3,7,2,102,6,Home,Away,First,Second,Aug,2
4,6,8,46,5,Home,Away,First,Second,Aug,6
...,...,...,...,...,...,...,...,...,...,...
2503,5,15,89,3,Neutral,Neutral,Second,First,Jun,5
2504,1,14,138,0,Home,Away,First,Second,Jan,1
2505,14,5,135,15,Home,Away,Second,First,Apr,14
2506,15,14,64,16,Home,Away,First,Second,Nov,14


In [82]:
test=pd.read_csv(path1)
test

,Team1,Team2,Stadium,HostCountry,Team1_Venue,Team2_Venue,Team1_Innings,Team2_Innings,MonthOfMatch
0,2,4,34,1,Home,Away,First,Second,Oct
1,14,1,19,15,Home,Away,First,Second,Mar
2,9,10,130,14,Neutral,Neutral,Second,First,Dec
3,9,10,8,9,Home,Away,First,Second,Dec
4,5,15,130,14,Neutral,Neutral,First,Second,Oct
...,...,...,...,...,...,...,...,...,...
1070,15,5,64,16,Home,Away,First,Second,Jun
1071,1,12,95,0,Home,Away,First,Second,Feb
1072,5,10,43,1,Neutral,Neutral,First,Second,Mar
1073,10,13,111,0,Neutral,Neutral,Second,First,Mar


In [83]:
#train.drop('Unnamed: 0',1,inplace=True)
#test.drop('Unnamed: 0',1,inplace=True)

In [84]:
train.columns

Index(['Team1', 'Team2', 'Stadium', 'HostCountry', 'Team1_Venue',
       'Team2_Venue', 'Team1_Innings', 'Team2_Innings', 'MonthOfMatch',
       'MatchWinner'],
      dtype='object')

In [87]:
#mean frequency encoding
def mean_encode(train_data, test_data, columns, target_col, reg_method=None,
                alpha=5, add_random=False, rmean=0, rstd=0.1, folds=1):
    '''Returns a DataFrame with encoded columns'''
    encoded_cols = []
    target_mean_global = train_data[target_col].mean()
    for col in columns:
        # Getting means for test data
        nrows_cat = train_data.groupby(col)[target_col].count()
        target_means_cats = train_data.groupby(col)[target_col].mean()
        target_means_cats_adj = (target_means_cats*nrows_cat + 
                                 target_mean_global*alpha)/(nrows_cat+alpha)
        # Mapping means to test data
        encoded_col_test = test_data[col].map(target_means_cats_adj)
        # Getting a train encodings
        if reg_method == 'expanding_mean':
            train_data_shuffled = train_data.sample(frac=1, random_state=1)
            cumsum = train_data_shuffled.groupby(col)[target_col].cumsum() - train_data_shuffled[target_col]
            cumcnt = train_data_shuffled.groupby(col).cumcount()
            encoded_col_train = cumsum/(cumcnt)
            encoded_col_train.fillna(target_mean_global, inplace=True)
            if add_random:
                encoded_col_train = encoded_col_train + normal(loc=rmean, scale=rstd, 
                                                               size=(encoded_col_train.shape[0]))
        elif (reg_method == 'k_fold') and (folds > 1):
            kfold = KFold( n_splits = 10, shuffle = False)
            parts = []
            for tr_in, val_ind in kfold.split(train_data):
                # divide data
                df_for_estimation, df_estimated = train_data.iloc[tr_in], train_data.iloc[val_ind]
                # getting means on data for estimation (all folds except estimated)
                nrows_cat = df_for_estimation.groupby(col)[target_col].count()
                target_means_cats = df_for_estimation.groupby(col)[target_col].mean()
                target_means_cats_adj = (target_means_cats*nrows_cat + 
                                         target_mean_global*alpha)/(nrows_cat+alpha)
                # Mapping means to estimated fold
                encoded_col_train_part = df_estimated[col].map(target_means_cats_adj)
                if add_random:
                    encoded_col_train_part = encoded_col_train_part + normal(loc=rmean, scale=rstd, 
                                                                             size=(encoded_col_train_part.shape[0]))
                # Saving estimated encodings for a fold
                
                parts.append(encoded_col_train_part)
            encoded_col_train = pd.concat(parts, axis=0)
            encoded_col_train.fillna(target_mean_global, inplace=True)
        else:
            encoded_col_train = train_data[col].map(target_means_cats_adj)
            if add_random:
                encoded_col_train = encoded_col_train + normal(loc=rmean, scale=rstd, 
                                                               size=(encoded_col_train.shape[0]))

        # Saving the column with means
        encoded_col = pd.concat([encoded_col_train, encoded_col_test], axis=0)
        encoded_col[encoded_col.isnull()] = target_mean_global
        encoded_cols.append(pd.DataFrame({'mean_'+target_col+'_'+col:encoded_col}))
    all_encoded = pd.concat(encoded_cols, axis=1)
    return all_encoded

In [88]:
cols = ['Team1', 'Team2', 'Stadium', 'HostCountry', 'Team1_Venue',
       'Team2_Venue', 'Team1_Innings', 'Team2_Innings', 'MonthOfMatch']
target = 'MatchWinner'
finaldf = mean_encode(train,test,cols,target,reg_method='k_fold',folds=5)

In [89]:
finaldf

,mean_MatchWinner_Team1,mean_MatchWinner_Team2,mean_MatchWinner_Stadium,mean_MatchWinner_HostCountry,mean_MatchWinner_Team1_Venue,mean_MatchWinner_Team2_Venue,mean_MatchWinner_Team1_Innings,mean_MatchWinner_Team2_Innings,mean_MatchWinner_MonthOfMatch
0,7.13,5.94,5.71,6.91,7.80,7.80,8.03,8.03,8.55
1,3.86,10.41,8.22,8.22,8.45,8.45,7.95,7.95,6.99
2,8.87,9.59,7.81,8.43,7.80,7.80,7.95,7.95,7.62
3,9.40,8.42,8.75,8.59,7.80,7.80,7.95,7.95,7.93
4,7.55,7.36,7.60,7.05,7.80,7.80,7.95,7.95,7.93
...,...,...,...,...,...,...,...,...,...
1070,9.85,6.58,9.93,9.74,7.82,7.82,7.97,7.97,7.14
1071,3.92,9.30,5.50,5.70,7.82,7.82,7.97,7.97,7.70
1072,7.07,8.02,7.60,7.04,8.43,8.43,7.97,7.97,7.89
1073,10.62,9.34,6.29,5.70,8.43,8.43,8.01,8.01,7.89


In [90]:
train_encoded=finaldf.iloc[0:2508,:]
test_encoded = finaldf.iloc[2508:,:]

In [91]:
train_encoded=pd.concat([train_encoded,y],1)
train_encoded

,mean_MatchWinner_Team1,mean_MatchWinner_Team2,mean_MatchWinner_Stadium,mean_MatchWinner_HostCountry,mean_MatchWinner_Team1_Venue,mean_MatchWinner_Team2_Venue,mean_MatchWinner_Team1_Innings,mean_MatchWinner_Team2_Innings,mean_MatchWinner_MonthOfMatch,MatchWinner
0,7.13,5.94,5.71,6.91,7.80,7.80,8.03,8.03,8.55,4
1,3.86,10.41,8.22,8.22,8.45,8.45,7.95,7.95,6.99,1
2,8.87,9.59,7.81,8.43,7.80,7.80,7.95,7.95,7.62,9
3,9.40,8.42,8.75,8.59,7.80,7.80,7.95,7.95,7.93,2
4,7.55,7.36,7.60,7.05,7.80,7.80,7.95,7.95,7.93,6
...,...,...,...,...,...,...,...,...,...,...
2503,7.11,9.47,9.16,6.48,8.33,8.33,8.00,8.00,7.26,5
2504,3.89,10.40,4.57,5.68,7.82,7.82,7.94,7.94,7.29,1
2505,10.79,6.62,9.58,9.97,7.82,7.82,8.00,8.00,8.65,14
2506,9.78,10.40,9.67,9.66,7.82,7.82,7.94,7.94,8.92,14


In [92]:
test_encoded

,mean_MatchWinner_Team1,mean_MatchWinner_Team2,mean_MatchWinner_Stadium,mean_MatchWinner_HostCountry,mean_MatchWinner_Team1_Venue,mean_MatchWinner_Team2_Venue,mean_MatchWinner_Team1_Innings,mean_MatchWinner_Team2_Innings,mean_MatchWinner_MonthOfMatch
0,6.82,5.85,7.05,7.04,7.82,7.82,7.97,7.97,8.64
1,10.69,4.69,8.06,9.87,7.82,7.82,7.97,7.97,7.89
2,8.92,8.02,8.80,8.64,8.43,8.43,8.01,8.01,8.49
3,8.92,8.02,7.94,8.42,7.82,7.82,7.97,7.97,8.49
4,7.07,9.46,8.80,8.64,8.43,8.43,7.97,7.97,8.64
...,...,...,...,...,...,...,...,...,...
1070,9.85,6.58,9.93,9.74,7.82,7.82,7.97,7.97,7.14
1071,3.92,9.30,5.50,5.70,7.82,7.82,7.97,7.97,7.70
1072,7.07,8.02,7.60,7.04,8.43,8.43,7.97,7.97,7.89
1073,10.62,9.34,6.29,5.70,8.43,8.43,8.01,8.01,7.89


In [93]:
from tqdm.notebook import tqdm 
from sklearn.metrics import log_loss
X=train_encoded.drop('MatchWinner',1)
y=train_encoded['MatchWinner']

X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.3,random_state=9)


lr=LogisticRegression(random_state=9)
dt=DecisionTreeClassifier(random_state=9)
rf=RandomForestClassifier(random_state=9)
lg=LGBMClassifier(random_state=9)
xgb=XGBClassifier(random_state=9)
catb=CatBoostClassifier(random_state=9)

models=[lr,dt,rf,lg,xgb,catb]


for model in tqdm(models):
    model.fit(X_train,y_train)
    y_pred=model.predict_proba(X_test)
    LOG_LOSS=log_loss(y_test,model.predict_proba(X_test))
    print('LOG LOSS:',LOG_LOSS)
    cv=cross_val_score(model,X,y,scoring='neg_log_loss',cv=5)
    print('mean score:',cv.mean())
    print('std:',cv.std())
    print('------------------')

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=9, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

LOG LOSS: 2.034617624409328
mean score: -2.0324158343676153
std: 0.03949442212548452
------------------


DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='gini',
                       max_depth=None, max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=9, splitter='best')

LOG LOSS: 15.92179852870834
mean score: -18.088265138650733
std: 0.9280083560903712
------------------


RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=9, verbose=0,
                       warm_start=False)

...

LGBMClassifier(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
               importance_type='split', learning_rate=0.1, max_depth=-1,
               min_child_samples=20, min_child_weight=0.001, min_split_gain=0.0,
               n_estimators=100, n_jobs=-1, num_leaves=31, objective=None,
               random_state=9, reg_alpha=0.0, reg_lambda=0.0, silent=True,
               subsample=1.0, subsample_for_bin=200000, subsample_freq=0)

LOG LOSS: 1.1158925330025091
mean score: -1.477126247669222
std: 0.1373773666665937
------------------


XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0,
              learning_rate=0.1, max_delta_step=0, max_depth=3,
              min_child_weight=1, missing=None, n_estimators=100, n_jobs=1,
              nthread=None, objective='multi:softprob', random_state=9,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=None, subsample=1, verbosity=1)

LOG LOSS: 0.9133000198855581



KeyboardInterrupt: 